In [1]:
import os
import json

CONFIG_DIR = "../.human_eval_config"
RESDULTS_DIR = "../results"

In [11]:
import zipfile
import datetime

os.makedirs(".backup", exist_ok=True)
current_time = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

# Backing up the Configs and Results
with zipfile.ZipFile(os.path.join("Backups", f"config_{current_time}"), 'w') as zipf:
    for root, dirs, files in os.walk(CONFIG_DIR):
        for file in files:
            zipf.write(os.path.join(root, file))

with zipfile.ZipFile(os.path.join("Backups", f"results_{current_time}"), 'w') as zipf:
    for root, dirs, files in os.walk(RESDULTS_DIR):
        for file in files:
            zipf.write(os.path.join(root, file))

In [2]:
with open(os.path.join(CONFIG_DIR, "config.json"), "r") as f:
    config = json.load(f)

In [4]:
results_files = [f for f in os.listdir(RESDULTS_DIR) if f.endswith(".json")]

workers = []
question_sets = []
completed_question_sets = []
incomplete_question_sets = []
for f in results_files:
    with open(os.path.join(RESDULTS_DIR, f)) as file:
        results = json.load(file)
        workers.append(results["worker_id"])
        question_sets.append(results["question_set_id"])
        if results["question_index"] == config["config"]["n_questions_per_worker"]:
            completed_question_sets.append(results["question_set_id"])
        else:
            incomplete_question_sets.append(results["question_set_id"])

In [6]:
len(set(completed_question_sets)), len(set(incomplete_question_sets)), len(workers), len(set(workers)), len(question_sets), len(set(question_sets))

(202, 34, 242, 229, 242, 226)

In [10]:
# list the items in completed_question_sets with count > 1
from collections import Counter
question_set_counts = Counter(incomplete_question_sets)
question_set_counts = {k: v for k, v in question_set_counts.items() if v > 1}
question_set_counts

{'32fa0bb6-3cbb-4b6a-90ba-a4dcc181a2b4': 2,
 '9419d52b-2d60-4f0f-bfd9-90a167e7ecc0': 2,
 '14617788-1c1b-42b2-8e95-e4422caae387': 2,
 'cd2a2d5e-d8e0-4bf0-99ed-daaebe9eb398': 2,
 '45a5e017-bd6a-4e69-8929-2ef5f842cd1d': 2}